In [10]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import SparsePCA
from sklearn.manifold import TSNE 
from sklearn.decomposition import TruncatedSVD

from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

from sklearn.pipeline import make_pipeline


In [3]:
subt = [line.rstrip('\n').replace('\\n',' ').replace('>','') for line in open('subtitles.txt')]

In [5]:
df = pd.DataFrame(columns=['context', 'reply'])
df['context'] = subt
df['context'] = df['context'].apply(lambda x: x.lower())
df['reply'] = subt[1:] + ['...']
df['reply'] = df['reply'].apply(lambda x: x.lower())

In [6]:
for sign in ['!', '?', ',', '.', ':']:
    df['context'] = df['context'].apply(lambda x: x.replace(sign,' '))
    df['reply'] = df['reply'].apply(lambda x: x.replace(sign,' '))

In [7]:
df.head(5)

,context,reply
0,obey the rules and manners to have a fun duel,light so glaring that it can't be hidden shatt...
1,light so glaring that it can't be hidden shatt...,the violently quaking and thirsty earth stakes...
2,the violently quaking and thirsty earth stakes...,just one more time is enough to create a miracle
3,just one more time is enough to create a miracle,for the power to regain the pages of memory lo...
4,for the power to regain the pages of memory lo...,the believing heart that penetrates the darkne...


In [8]:

vectorizer = TfidfVectorizer()
vectorizer.fit(df.context)
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
vector = vectorizer.transform(df.head(10))
print(vector.shape)
print(vector.toarray())
vector_large = vectorizer.transform(df.context)

{'obey': 36190, 'the': 52140, 'rules': 43999, 'and': 3011, 'manners': 31434, 'to': 52859, 'have': 22277, 'fun': 19341, 'duel': 14898, 'light': 30040, 'so': 47981, 'glaring': 20317, 'that': 52066, 'it': 26349, 'can': 8079, 'be': 5471, 'hidden': 22949, 'shatters': 46176, 'silence': 47165, 'revives': 43216, 'in': 24945, 'this': 52373, 'world': 57910, 'violently': 56239, 'quaking': 41145, 'thirsty': 52367, 'earth': 15123, 'stakes': 49114, 'its': 26430, 'life': 29994, 'clashes': 9575, 'together': 52942, 'with': 57692, 'sparks': 48527, 'just': 27172, 'one': 36784, 'more': 33782, 'time': 52680, 'is': 26159, 'enough': 16099, 'create': 11547, 'miracle': 33032, 'for': 18632, 'power': 39941, 'regain': 42260, 'pages': 37897, 'of': 36405, 'memory': 32323, 'lost': 30571, 'past': 38264, 'believing': 5776, 'heart': 22451, 'penetrates': 38522, 'darkness': 12332, 'searches': 45375, 'place': 39265, 'where': 57294, 'soul': 48376, 'sleeps': 47570, 'burns': 7655, 'sharp': 46159, 'glance': 20307, 'eyes': 171

In [11]:
print(vector_large.shape)
TS = TruncatedSVD(n_components=300, algorithm='randomized')

TS.fit(vector_large)
vector_small = TS.transform(vector_large)

# Print new dimensionality and explained variance ratio
print(vector_small.shape)



(1248751, 59407)
(1248751, 300)


In [28]:
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

class NeighborSampler(BaseEstimator):
    def __init__(self, k=10, temperature = 2.0):
        self.k = k
        self.temperature = temperature
    
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
        
    def predict(self, X, random_state = None):
        dis, ind = self.tree_.query(X, return_distance = True, k = self.k)
        result = []
        for d, i in zip(dis, ind):
            result.append(np.random.choice(i, p = sigmoid(d * self.temperature)))
            
        return self.y_[result]

In [29]:
nei_sam = NeighborSampler()
nei_sam.fit(vector_small, df.reply)

pipe = make_pipeline(vectorizer, TS, nei_sam)

In [48]:
print('To stop chatting type \'BYE(All uppercase)\'')
x= input()
while x!='BYE':
      print('JenBo: ',pipe.predict([x]))
      x=input('You: ')

To stop chatting type 'BYE(All uppercase)'
Hi
JenBo:  ['thanks  nako ']
You: How do you feel
JenBo:  ['the seed i shot into you will soon take root and control your body ']
You: you are rude
JenBo:  ['g-good grief  what the hell are  you talking about  polnareff ']
You: Do you know about haki?
JenBo:  ['no  would you like me to investigate ']
You: yes
JenBo:  ['take care on your trip ']
You: nice prank
JenBo:  ['tanaka is scoring consistently  though ']
You: do you want meat? 
JenBo:  ['no ']
You: why?
JenBo:  ["why does it matter  so you're doing something for christmas "]
You: BYE
